# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [52]:
!pip install tensorflow==2.0.0

In [0]:
from keras.datasets import cifar10

In [54]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
(train_x, train_y),(test_x,test_y) = tf.keras.datasets.cifar10.load_data()

In [81]:
print ( train_x.shape)
print ( test_x.shape)
print ( train_y.shape)
print ( test_y.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [82]:
y04 = np.where(train_y <= 4) 
x04 = train_x[y04[0]]
x04.shape 

(25000, 32, 32, 3)

In [83]:
y59 = np.where(train_y >= 5)
x59= train_x[y59[0]] 
x59.shape

(25000, 32, 32, 3)

In [60]:
ytest04 = np.where(test_y <= 4)
xtest04= test_x[ytest04[0]] 
xtest04.shape 

(5000, 32, 32, 3)

In [79]:
ytest59 = np.where(test_y >= 5)
xtest59 = test_x[ytest59[0]] 
xtest59.shape

(5000, 32, 32, 3)

In [85]:
train04y = train_y[y04]
test04y= test_y[ytest04]
test04y.shape

(5000,)

In [86]:
train59y = train_y[y59]
test59y= test_y[ytest59]
test59y.shape

(5000,)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [88]:
y04_ = tf.keras.utils.to_categorical(train04y)
testy04_ = tf.keras.utils.to_categorical(test04y)
y04_.shape

(25000, 5)

In [0]:
y59_ = tf.keras.utils.to_categorical(train59y,num_classes=10)
testy59_ = tf.keras.utils.to_categorical(test59y,num_classes =10)



In [92]:
y59new = y59_[:,5:10] 
y59new.shape
testy59new = testy59_[:,5:10] 
testy59new.shape

(5000, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout,Activation
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import Accuracy

In [0]:
tf.keras.backend.clear_session()
# Sequential
model = Sequential()

# 1st CNN layer
model.add(Conv2D(32, (3, 3), padding='same',input_shape=x04.shape[1:]))
model.add(Activation('relu'))

# Max pool layer with 20% dropout
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20)) 

# 2nd CNN layer 
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

# Batcth normalizing
model.add(BatchNormalization())

# 3rd CNN layer 
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

# 4th CNN layer 
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

# max pool  2 with 30% dropout
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.30)) 

# Flattening the layer 
model.add(Flatten())

# Dens layer 
model.add(Dense(512,activation ='relu'))

# Output layer 
model.add(Dense(5,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [102]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 32)        1

In [0]:
mckpt = tf.keras.callbacks.ModelCheckpoint('./cifar.h5', 
                                           monitor='val_acc', save_best_only=True,)

In [104]:
history = model.fit(x04,y04_,          
          validation_data=(xtest04,testy04_),
          epochs=10,
          batch_size=32, callbacks=[mckpt],workers=2)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 97s 4ms/sample - loss: 1.0544 - accuracy: 0.5713 - val_loss: 1.1122 - val_accuracy: 0.5748
Epoch 2/10
25000/25000 [==============================] - 96s 4ms/sample - loss: 0.8406 - accuracy: 0.6668 - val_loss: 0.8111 - val_accuracy: 0.6756
Epoch 3/10
25000/25000 [==============================] - 96s 4ms/sample - loss: 0.7414 - accuracy: 0.7130 - val_loss: 0.7057 - val_accuracy: 0.7316
Epoch 4/10
25000/25000 [==============================] - 96s 4ms/sample - loss: 0.6676 - accuracy: 0.7428 - val_loss: 0.6587 - val_accuracy: 0.7420
Epoch 5/10
25000/25000 [==============================] - 96s 4ms/sample - loss: 0.6124 - accuracy: 0.7674 - val_loss: 0.6653 - val_accuracy: 0.7520
Epoch 6/10
25000/25000 [==============================] - 97s 4ms/sample - loss: 0.5525 - accuracy: 0.7900 - val_loss: 0.6113 - val_accuracy: 0.7642
Epoch 7/10
25000/25000 [==============================] -

In [111]:
history = model.fit(x04,y04_,        
          validation_data=(xtest04,testy04_),
          epochs=15,
          initial_epoch=10,
          batch_size=32, callbacks=[mckpt],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 11/15
25000/25000 [==============================] - 95s 4ms/sample - loss: 0.3583 - accuracy: 0.8665 - val_loss: 0.6893 - val_accuracy: 0.7708
Epoch 12/15
25000/25000 [==============================] - 95s 4ms/sample - loss: 0.3288 - accuracy: 0.8773 - val_loss: 0.7905 - val_accuracy: 0.7400
Epoch 13/15
25000/25000 [==============================] - 95s 4ms/sample - loss: 0.2956 - accuracy: 0.8909 - val_loss: 0.5893 - val_accuracy: 0.8042
Epoch 14/15
25000/25000 [==============================] - 96s 4ms/sample - loss: 0.2841 - accuracy: 0.8958 - val_loss: 0.7029 - val_accuracy: 0.7810
Epoch 15/15
25000/25000 [==============================] - 97s 4ms/sample - loss: 0.2552 - accuracy: 0.9067 - val_loss: 0.6176 - val_accuracy: 0.7982


In [123]:
history = model.fit(x04,y04_,        
          validation_data=(xtest04,testy04_),
          epochs=20,
          initial_epoch=15,
          batch_size=32, callbacks=[mckpt],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 16/20
25000/25000 [==============================] - 91s 4ms/sample - loss: 0.2461 - accuracy: 0.9107 - val_loss: 0.6113 - val_accuracy: 0.7998
Epoch 17/20
25000/25000 [==============================] - 92s 4ms/sample - loss: 0.2267 - accuracy: 0.9185 - val_loss: 0.6707 - val_accuracy: 0.8024
Epoch 18/20
25000/25000 [==============================] - 91s 4ms/sample - loss: 0.2107 - accuracy: 0.9246 - val_loss: 0.6379 - val_accuracy: 0.8080
Epoch 19/20
25000/25000 [==============================] - 92s 4ms/sample - loss: 0.1994 - accuracy: 0.9280 - val_loss: 0.6325 - val_accuracy: 0.8032
Epoch 20/20
25000/25000 [==============================] - 91s 4ms/sample - loss: 0.1834 - accuracy: 0.9348 - val_loss: 0.6460 - val_accuracy: 0.8008


In [0]:
#accuracy of 80.32 achieved

In [0]:
model.save_weights('weight.hdf5')

In [0]:
model.save('model.hdf5') 

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [0]:
model59 =  load_model('model.hdf5')
for layer in model59.layers[:13]:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

In [116]:
model59.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model59.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 32)        1

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
mckpt59 = tf.keras.callbacks.ModelCheckpoint('./cifar59.h5', 
                                           monitor='val_acc', save_best_only=True,)

In [121]:
history59 = model59.fit(x59,y59new,          
          validation_data=(xtest59,testy59new),
          epochs=10,
          batch_size=32, callbacks=[mckpt59],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 39s 2ms/sample - loss: 1.0054 - accuracy: 0.6756 - val_loss: 0.6143 - val_accuracy: 0.7712
Epoch 2/10
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.5484 - accuracy: 0.7986 - val_loss: 0.5199 - val_accuracy: 0.8074
Epoch 3/10
25000/25000 [==============================] - 39s 2ms/sample - loss: 0.4519 - accuracy: 0.8361 - val_loss: 0.5030 - val_accuracy: 0.8174
Epoch 4/10
25000/25000 [==============================] - 39s 2ms/sample - loss: 0.3903 - accuracy: 0.8578 - val_loss: 0.5249 - val_accuracy: 0.8176
Epoch 5/10
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.3290 - accuracy: 0.8810 - val_loss: 0.5234 - val_accuracy: 0.8248
Epoch 6/10
25000/25000 [==============================] - 39s 2ms/sample - loss: 0.2828 - accuracy: 0.8968 - val_loss: 0.5168 - val_accuracy: 0.8238
Epoch 7/10
25000/25000 [==============================] -

In [122]:
history59 = model59.fit(x59,y59new,          
          validation_data=(xtest59,testy59new),
          epochs=15,
          initial_epoch=10,
          batch_size=32, callbacks=[mckpt59],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 11/15
25000/25000 [==============================] - 39s 2ms/sample - loss: 0.1470 - accuracy: 0.9481 - val_loss: 0.5811 - val_accuracy: 0.8314
Epoch 12/15
25000/25000 [==============================] - 39s 2ms/sample - loss: 0.1332 - accuracy: 0.9525 - val_loss: 0.6323 - val_accuracy: 0.8230
Epoch 13/15
25000/25000 [==============================] - 39s 2ms/sample - loss: 0.1344 - accuracy: 0.9516 - val_loss: 0.6361 - val_accuracy: 0.8292
Epoch 14/15
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.1217 - accuracy: 0.9571 - val_loss: 0.6292 - val_accuracy: 0.8312
Epoch 15/15
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.1149 - accuracy: 0.9601 - val_loss: 0.6376 - val_accuracy: 0.8296


In [124]:
history59 = model59.fit(x59,y59new,          
          validation_data=(xtest59,testy59new),
          epochs=25,
          initial_epoch=15,
          batch_size=32, callbacks=[mckpt59],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 16/25
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.1095 - accuracy: 0.9607 - val_loss: 0.6471 - val_accuracy: 0.8312
Epoch 17/25
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.0986 - accuracy: 0.9665 - val_loss: 0.6701 - val_accuracy: 0.8318
Epoch 18/25
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.0989 - accuracy: 0.9657 - val_loss: 0.6852 - val_accuracy: 0.8262
Epoch 19/25
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.0887 - accuracy: 0.9695 - val_loss: 0.7159 - val_accuracy: 0.8256
Epoch 20/25
25000/25000 [==============================] - 39s 2ms/sample - loss: 0.0857 - accuracy: 0.9702 - val_loss: 0.6648 - val_accuracy: 0.8360
Epoch 21/25
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.0860 - accuracy: 0.9708 - val_loss: 0.7047 - val_accuracy: 0.8334
Epoch 22/25
25000/25000 [==========================

In [125]:
history59 = model59.fit(x59,y59new,          
          validation_data=(xtest59,testy59new),
          epochs=30,
          initial_epoch=25,
          batch_size=32, callbacks=[mckpt59],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 26/30
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.0739 - accuracy: 0.9757 - val_loss: 0.7686 - val_accuracy: 0.8330
Epoch 27/30
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.0663 - accuracy: 0.9778 - val_loss: 0.7689 - val_accuracy: 0.8354
Epoch 28/30
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.0721 - accuracy: 0.9749 - val_loss: 0.7512 - val_accuracy: 0.8372
Epoch 29/30
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.0695 - accuracy: 0.9771 - val_loss: 0.8095 - val_accuracy: 0.8370
Epoch 30/30
25000/25000 [==============================] - 38s 2ms/sample - loss: 0.0650 - accuracy: 0.9780 - val_loss: 0.7586 - val_accuracy: 0.8356


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [66]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [67]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [68]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline

In [0]:
# instantiate the vectorizer
vect = TfidfVectorizer()

In [0]:
vect = TfidfVectorizer(stop_words='english')
dtm_train = vect.fit_transform(twenty_train.data)
dtm_test = vect.transform(twenty_test.data)
features = vect.get_feature_names()

In [78]:
print(vect.get_feature_names()[-10:])

['zvi', 'zvonko', 'zwart', 'zyeh', 'zyklon', 'zyxel', 'zz', 'zzz', 'ªl', 'íålittin']


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
logreg = LogisticRegression()

In [74]:
logreg.fit(dtm_train, twenty_train.target)
y_pred_class = logreg.predict(dtm_test)
print('test Accuracy ',metrics.accuracy_score(twenty_test.target, y_pred_class))
y_pred_class_train = logreg.predict(dtm_train)
print('train accuracy ',metrics.accuracy_score(twenty_train.target, y_pred_class_train))

test Accuracy  0.9054593874833555
train accuracy  0.9968985378821444


In [75]:
# Classification report  
print(metrics.classification_report(twenty_test.target, y_pred_class))

              precision    recall  f1-score   support

           0       0.96      0.76      0.85       319
           1       0.87      0.98      0.92       389
           2       0.94      0.90      0.92       396
           3       0.88      0.95      0.91       398

    accuracy                           0.91      1502
   macro avg       0.91      0.90      0.90      1502
weighted avg       0.91      0.91      0.90      1502

